In [312]:
from openai import OpenAI
import pandas as pd
import IPython.display as ipd
import random
import json
import numpy as np
import os
from collections import defaultdict
import random
from eedi.datasets import make_nice_df, make_complete_query
from pydantic import BaseModel
from dotenv import load_dotenv
from pydantic.alias_generators import to_pascal
import seaborn as sns
load_dotenv()

True

In [329]:
df_train = pd.read_csv("../data/train.csv")
df_train = make_nice_df(df_train)
# df_train["QuestionComplete"] = df_train.apply(make_complete_query, axis=1)
df_mis = pd.read_csv("../data/misconception_mapping.csv")
orig_mis = df_mis["MisconceptionName"]
df_train["Synthetic"] = False

with open("../data/synthetic/synthetic_known.json", "r") as f:
    synthetic_known = json.load(f)

with open("../data/synthetic/synthetic_unknown.json", "r") as f:
    synthetic_unknown = json.load(f)

In [309]:
df_train.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectChoice,CorrectText,QuestionText,WrongChoice,WrongText,MisconceptionId,QuestionId_Answer,Synthetic
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\( 3 \times(2+4)-5 \),\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,D,Does not need brackets,1672,0_D,False
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",A,\( m+1 \),2142,1_A,False
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",B,\( m+2 \),143,1_B,False
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",C,\( m-1 \),2142,1_C,False
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Only\nKatie,Tom and Katie are discussing the \( 5 \) plant...,A,Only\nTom,1287,2_A,False


In [ ]:
subject_to_subject_id = {}
for i, s_name, s_id in df_train[["SubjectName", "SubjectId"]].itertuples():
    subject_to_subject_id[s_name] = s_id

construct_to_construct_id = {}
for i, c_name, c_id in df_train[["ConstructName", "ConstructId"]].itertuples():
    construct_to_construct_id[c_name] = c_id

question_to_question_id = {}
for i, q_text, q_id in df_train[["QuestionText", "QuestionId"]].itertuples():
    question_to_question_id[q_text] = q_id

mis_to_mis_id = {mis_text: mis_id for mis_id, mis_text in enumerate(orig_mis)}

In [323]:
df_train.columns

Index(['QuestionId', 'ConstructId', 'ConstructName', 'SubjectId',
       'SubjectName', 'CorrectChoice', 'CorrectText', 'QuestionText',
       'WrongChoice', 'WrongText', 'MisconceptionId', 'QuestionId_Answer',
       'Synthetic'],
      dtype='object')

In [332]:
# rebuild
new_train_rows = []
for known in synthetic_known:
    new_q = known["question"]
    # NOTE: this is bad slow code but i dont care
    if new_q not in question_to_question_id:
        question_to_question_id[new_q] = max(question_to_question_id.values()) + 1
    new_train_rows.append(
        {
            "QuestionId": question_to_question_id[new_q],
            "ConstructId": construct_to_construct_id[known["construct_name"]],
            "ConstructName": known["construct_name"],
            "SubjectId": subject_to_subject_id[known["subject_name"]],
            "SubjectName": known["subject_name"],
            "CorrectChoice": "?",
            "CorrectText": known["correct_answer"],
            "QuestionText": new_q,
            "WrongChoice": "?",
            "WrongText": known["wrong_answer"],
            "MisconceptionId": mis_to_mis_id[known["misconception_name"]],
            "QuestionId_Answer": "?",
            "Synthetic": True,
        }
    )
for unknown in synthetic_unknown:
    # NOTE: these are bad slow code but i dont care
    new_subj = unknown["subject_name"]
    new_c = unknown["construct_name"]
    new_q = unknown["question"]
    if new_subj not in subject_to_subject_id:
        subject_to_subject_id[new_subj] = max(subject_to_subject_id.values()) + 1
    if new_c not in construct_to_construct_id:
        construct_to_construct_id[new_c] = max(construct_to_construct_id.values()) + 1
    if new_q not in question_to_question_id:
        question_to_question_id[new_q] = max(question_to_question_id.values()) + 1
    new_train_rows.append(
        {
            "QuestionId": question_to_question_id[new_q],
            "ConstructId": construct_to_construct_id[new_c],
            "ConstructName": new_c,
            "SubjectId": subject_to_subject_id[new_subj],
            "SubjectName": new_subj,
            "CorrectChoice": "?",
            "CorrectText": unknown["correct_answer"],
            "QuestionText": new_q,
            "WrongChoice": "?",
            "WrongText": unknown["wrong_answer"],
            "MisconceptionId": mis_to_mis_id[unknown["misconception_name"]],
            "QuestionId_Answer": "?",
            "Synthetic": True,
        }
    )

In [338]:
df_train_new = pd.DataFrame.from_records(new_train_rows)

df_train_all = pd.concat([df_train, df_train_new]).reset_index(drop=True)
df_train_all

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectChoice,CorrectText,QuestionText,WrongChoice,WrongText,MisconceptionId,QuestionId_Answer,Synthetic
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\( 3 \times(2+4)-5 \),\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,D,Does not need brackets,1672,0_D,False
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",A,\( m+1 \),2142,1_A,False
2,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",B,\( m+2 \),143,1_B,False
3,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,Does not simplify,"Simplify the following, if possible: \( \frac{...",C,\( m-1 \),2142,1_C,False
4,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Only\nKatie,Tom and Katie are discussing the \( 5 \) plant...,A,Only\nTom,1287,2_A,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35877,30891,11125,Understand reflection over a line,3340,Reflection in Geometry,?,"(-3, 4)",Which of the following points is the reflectio...,?,"(3, -4)",2041,?,True
35878,30892,11126,Identify line of symmetry,3316,Symmetry,?,x = 0,Which of the following lines is the line of sy...,?,y = 0,2041,?,True
35879,30893,11127,Understanding reflection over x-axis,3341,Basic Transformations,?,"(5, 7)","Reflect the point (5, -7) over the x-axis. Wha...",?,"(-5, -7)",2041,?,True
35880,30894,11128,Reflect across a given line,2201,Graphing Linear Equations,?,"(3, 2)","If point A(2, 3) is reflected over the line y ...",?,"(-3, -2)",2041,?,True


In [331]:
synthetic_unknown[0]

{'subject_name': 'Standard Form and Scientific Notation',
 'construct_name': 'Convert numbers between standard form and ordinary form',
 'question': 'What is 3.45 x 10^4 in ordinary form?',
 'correct_answer': '34500',
 'wrong_answer': '0.000345',
 'misconception_name': 'Believes all numbers written in standard form must be smaller than any numbers written in ordinary form'}

In [328]:
len(new_train_rows)

21807

In [ ]:
# rebuilding dataset


{'subject_name': 'Averages (mean, median, mode) from a List of Data',
 'construct_name': 'Order fractions whose denominators are multiples of the same number ',
 'question': '\\( \\begin{array}{llllll}\\frac{9}{12} & \\frac{35}{48} & \\frac{5}{16} & 0.75 & \\frac{3}{4} & \\frac{65}{80}\\end{array} \\) Which value is the outlier in this set of data?',
 'correct_answer': '\\( \\frac{5}{16} \\)',
 'wrong_answer': '\\( \\frac{65}{80} \\)',
 'misconception_name': 'Believes an outlier is the value in a data set with the biggest denominator, even if this value is close to the mean'}